In [1]:
import h5py
import torch
import transformers
from tqdm import tqdm
from transformers import AutoModelForMaskedLM, AutoTokenizer
import sys
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
sys.path.append('../data_generation')
import utils
import numpy as np

/home/ztang/.conda/envs/gpn_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
datalen = '512'
file = h5py.File("../data/CAGI/"+datalen+"/CAGI_onehot.h5", "r")
alt = file['alt']
ref = file['ref']

## GPN claim zero shot learning with log p(alt)/p(ref)

Need to mask the center nuceltide of each seq and get the p value

In [3]:
import gpn
import gpn.model
model = AutoModelForMaskedLM.from_pretrained("../model/GPN_human/checkpoint-2000000").to('cuda')
tokenizer = AutoTokenizer.from_pretrained("../model/GPN_human/checkpoint-2000000")
model.eval();

In [6]:
N, L, A = alt.shape
mut_i = int(L/2-1)
cagi_llr=[]
for i in tqdm(range(N)):
    ref_a = np.argmax(ref[i][mut_i])+2
    alt_a = np.argmax(alt[i][mut_i])+2
    seq = utils.onehot_to_seq([ref[i]])
    token = tokenizer.batch_encode_plus(seq)['input_ids'][0]
    token[mut_i] = tokenizer.mask_token_id
    output = model(torch.tensor([token]).to('cuda')).logits.cpu().detach().numpy()
    llr = output[0][mut_i][alt_a] - output[0][mut_i][ref_a]
    cagi_llr.append(llr)


100%|██████████| 18442/18442 [02:22<00:00, 129.34it/s]


In [9]:
output = h5py.File('../data/CAGI/'+'cagi_'+datalen+'_'+'gpn.h5', 'w')
output.create_dataset('llr', data=np.array(cagi_llr))
output.close()

## Visulization


In [5]:
import h5py
import pandas as pd
import numpy as np
from operator import itemgetter
import seaborn as sns
import matplotlib.pyplot as plt 
import scipy.stats as stats
datalen='512'

In [6]:
cagi_result = h5py.File('../data/CAGI/'+'cagi_'+datalen+'_'+'gpn.h5', 'r')
cagi_llr = cagi_result['llr'][()]

In [8]:
cagi_df = pd.read_csv('../data/CAGI/'+datalen+'/final_cagi_metadata.csv',
                      index_col=0).reset_index()
target = cagi_df['6'].values.tolist()
exp_list = cagi_df['8'].unique()

In [10]:
perf = []
sanity_check = 0
for exp in ['LDLR','SORT1','F9','PKLR']:
    sub_df = cagi_df[cagi_df['8'] == exp]
    sanity_check += len(sub_df)
    exp_target = np.array(target)[sub_df.index.to_list()]
    exp_pred = np.squeeze(cagi_llr)[sub_df.index.to_list()]
    exp_target = np.absolute(exp_target)
    exp_pred = np.absolute(exp_pred)
    print(exp)
    perf.append(stats.pearsonr(exp_pred,exp_target)[0])
    print(stats.pearsonr(exp_pred,exp_target)[0])

LDLR
-0.07866246778179416
SORT1
-0.001525750065541454
F9
0.07505647712949828
PKLR
0.03675438661873984
